De forma geral será feito um agente que possui observações parciais do ambiente, tendo como objetivo encontrar um caminho até o objetivo.

Como característica do ambiente temos que ele é determinístico, desconhecido, parcialmente observável, suas ações serão dependentes (time-series), estático, suas percepções serão determinísticas e haverá apenas um agente no ambiente (single agent).

Com base nas características do ambiente será construído um agente, sendo um agente baseado em objetivos com representação atômica, ou seja, não haverá estrutura na qual o agente poderá inferir informações, sua única capacidade será, dado um estado, dizer se é o objetivo ou não, sendo esse tipo de agente é chamado busca, (ideal é que esse ambiente fosse totalmente conhecido, depois o tornaremos)

In [ ]:
import numpy as np

In [ ]:
"""Iremos criar a classe onde a parte abstrata da busca esrará ocorrendo.
Segundo o livro essa estrutura pode ser representada como uma árvore e que em cada nó deve conter o estado do nó 'estado' 
(Como complemoto ao estado adicionamos outro atributo 'local'), 
ação que formou o nó 'acao', o pai do nó atual 'parent', e o 'custo' da raiz até o nó atual.
E por fim além dos atributos precisamos de um método para avançar para os próximos estados 'expandir', no livro está descrito como 
'modelo de transição'"""

class node:

    def __init__(self, estado, local, acao=None, parent=None, custo=0):
        self.estado = estado
        self.acao = acao
        self.custo = custo
        self.local = local
        self.parent = parent
        
    def __repr__(self):
        return "<Node {}>".format(self.estado)
         
# Metodo utilizado para expandir (avançar) para os próximos estados
    def expandir(self, problema):

        return [self.no_filho(problema, acoes)
                for acoes in problema.acoes_pos(self.estado, self.local)]
    
# Esse método será utilizado pelo 'expandir' para realmente demostrar o novo estado
    def no_filho(self, problema, acao):

        proximo_estado = problema.resultado(self.estado, acao, self.local)
        
        if acao == 'cima':
            local = [self.local[0]-1, self.local[1]]
        elif acao == 'baixo':
            local = [self.local[0]+1,self.local[1]]
        elif acao == 'esquerda':
            local = [self.local[0],self.local[1]-1]
        else:
            local = [self.local[0],self.local[1]+1]

        prox_node = node(proximo_estado, local, acao, self, problema.custo_caminho(self.custo))

        return prox_node
    
    """Devido aos parametros inicializados com a classe podemos refazer os passos, então depois de 
    encontrado o objetivo, será feito um caminho inverso demostrando as ações tomadas"""
    def solucao(self):

        return [no.acao for no in self.cam()[::-1][1:]]
    
# Sendo cam o método que irá uxiliar a 'solucao' a fazer o caminho inverso
    def cam(self):

        nodee, ca = self, []
        while nodee:
            ca.append(nodee)
            nodee = nodee.parent
        return(list(ca))

In [ ]:
"""Aqui irá acontecer a definição em si do problema, suas regras, possíveis ações, resultado das ações, teste de objetivo, etc.
os metodos e sua construção irá depender muito do problema à ser abordado"""

class caminho:

    def __init__(self, inicial, objetivo):
        self.inital = inicial
        self.objetivo = objetivo
        dim = inicial.shape
        self.dim = [dim[0]-1, dim[1]-1]

    def custo_caminho(self, c):

        return c+1

    def acoes_pos(self, estado, local, cus):

        acoes = []

        """Variáveis utilizadas para localização caso o agente esteja cercado por 
        estados já. Permitindo passos redundantes"""
        cnt_sup_dir = 0
        cnt_sup_esq = 0
        cnt_inf_dir = 0
        cnt_inf_esq = 0
        row_sup = 0
        row_inf = 0
        col_esq = 0
        col_dir = 0

        # esta na primeira coluna
        if local[1] == 0:
            col_esq+=1

            # Canto superior esquerdo
            if local == [0,0]:
                cnt_sup_esq+=1

                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:
    
                    acoes.append('baixo')

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       
         
                    acoes.append('direita')

            # Canto inferior esquerdo
            elif local == [self.dim[0],0]:
                cnt_inf_esq+=1
                        
                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  

                    acoes.append('cima') 

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       
    
                    acoes.append('direita')

            else:
                col_esq+=1
                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:
            
                    acoes.append('baixo')

                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  
               
                    acoes.append('cima')

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       
          
                    acoes.append('direita')               
        
        # esta na primeira fileira
        elif local[0] == 0:
            row_sup+=1
                    
            # Canto superior direito
            if local == [0,self.dim[0]]:
                cnt_sup_dir+=1

                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:

                        acoes.append('baixo')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:
                        
                        acoes.append('esquerda')

            # Canto superior esquerdo
            elif local == [0,0]:
                cnt_sup_esq+=1

                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:

                    acoes.append('baixo')

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       

                    acoes.append('direita')

            else:
           
                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:

                    acoes.append('baixo')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:

                    acoes.append('esquerda')

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       

                    acoes.append('direita')

        # esta na ultima fileira
        elif local[0] == self.dim[0]:
            row_inf+=1

            # Canto inferior direito
            if local == [self.dim[0],self.dim[1]]:
                cnt_inf_dir+=1

                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  

                    acoes.append('cima')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:

                    acoes.append('esquerda')

            # Canto inferior esquerdo
            elif local == [self.dim[0],0]:
                cnt_inf_esq+=1

                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  

                    acoes.append('cima') 

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       
 
                    acoes.append('direita')

            else:

                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  

                    acoes.append('cima')
                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:
                  
                    acoes.append('esquerda')
                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       
                  
                    acoes.append('direita')

        # esta na ultima coluna
        elif local[1] == self.dim[1]:
            col_dir+=1
                                 
            # Canto inferior direito
            if local == [self.dim[0],self.dim[1]]:
                cnt_inf_dir+=1
     
                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  
              
                    acoes.append('cima')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:
                  
                    acoes.append('esquerda')

            # Canto superior direito
            elif local == [0,self.dim[1]]:
                cnt_sup_dir+=1

                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:
                   
                        acoes.append('baixo')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:
                   
                        acoes.append('esquerda')

            else:

                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:
            
                    acoes.append('baixo')

                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  
          
                    acoes.append('cima')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:

                    acoes.append('esquerda')

        else:
                   
            # testar se veio de baixo
            if estado[local[0]+1][local[1]] != 1:

                acoes.append('baixo')
            # testar se veio de cima
            if estado[local[0]-1][local[1]] != 1:  

                acoes.append('cima') 
            # testar se veio da esquerda
            if estado[local[0]][local[1]-1] != 1:

                acoes.append('esquerda')
            # testar se veio da direita
            if estado[local[0]][local[1]+1] != 1:
                
                acoes.append('direita')

        """Caso o agente estaja em um local sercado por 1, ou seja, que já tenham sido visitados
        e pode haver parede. Ele será permitido a fazer um passo redundante, avançando para um local já
        visitado, isso será possível com as variáveis iníciadas no começo do método"""

        if len(acoes) == 0:
            # Iremos adicionar 1 há uma lista, dessa forma iremos saber a quantia de passos redundantes
            cus.append(1)
          
            if row_sup >= 1:
                
                if cnt_sup_dir >= 1:
                    acoes = ['baixo', 'esquerda']
                elif cnt_sup_esq >= 1:
                    acoes = ['baixo', 'direita']
                else:
                    acoes = ['baixo', 'esquerda', 'direita']

            elif row_inf >= 1:

                if cnt_inf_dir >= 1:
                    acoes = ['cima', 'esquerda']

                elif cnt_inf_esq >= 1:
                    acoes = ['cima', 'direita']

                else:
                    acoes = ['cima', 'esquerda', 'direita']

            elif col_dir >= 1:

                if cnt_inf_dir >= 1:
                    acoes = ['cima', 'esquerda']

                elif cnt_sup_dir >= 1:
                    acoes = ['baixo', 'esquerda']

                else:
                    acoes = ['cima', 'baixo', 'esquerda']

            elif col_esq >= 1:

                if cnt_inf_esq >= 1:
                    acoes = ['cima', 'direita']

                elif cnt_sup_esq >= 1:
                    acoes = ['baixo', 'direita']

                else:
                    acoes = ['cima', 'baixo', 'direita']
                
            else:
                acoes = ['cima', 'baixo', 'esquerda', 'direita']

        return acoes